In [1]:
!pip install netCDF4
!pip install xarray
!pip install h5netcdf


  Using cached netCDF4-1.7.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached cftime-1.6.4.post1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.7 kB)
Using cached netCDF4-1.7.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (9.3 MB)
Using cached cftime-1.6.4.post1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.4 MB)
  Using cached xarray-2025.3.1-py3-none-any.whl.metadata (12 kB)
Using cached xarray-2025.3.1-py3-none-any.whl (1.3 MB)
  Using cached h5netcdf-1.6.1-py3-none-any.whl.metadata (13 kB)
Using cached h5netcdf-1.6.1-py3-none-any.whl (49 kB)


In [2]:
import boto3
from netCDF4 import Dataset
import io

# S3 client oluşturma
s3 = boto3.client('s3')

# Bucket ve dosya anahtarını (key) belirtin
bucket_name = "airlinessss"   # Örneğin: "airlinessss"
file_key = "2017_merged.nc"         # Dosya ismi, örneğin: "weather_data.nc"

# S3'ten dosyayı çekme
obj = s3.get_object(Bucket=bucket_name, Key=file_key)

# Dosyanın içeriğini bytes olarak okuma ve BytesIO nesnesine aktarma
data = obj['Body'].read()
nc_buffer = io.BytesIO(data)

# NetCDF dosyasını bellekten okuma
dataset = Dataset('inmemory.nc', mode='r', memory=nc_buffer.getvalue())

# Boyutların (dimensions) listesini görüntüleme
print("Boyutlar:")
for dim in dataset.dimensions:
    print(dim, ":", dataset.dimensions[dim].size)

# Değişkenlerin (variables) listesini görüntüleme
print("\nDeğişkenler:")
for var in dataset.variables:
    print(var)

# Belirli bir değişkenin verilerini (örneğin 'temperature') görüntüleme
if 'temperature' in dataset.variables:
    temperature_data = dataset.variables['temperature'][:]
    print("\nTemperature Değişkeninin Verileri:")
    print(temperature_data)
else:
    print("\n'temperature' isminde bir değişken bulunamadı.")

# Dataset'i kapatma
dataset.close()


Boyutlar:
valid_time : 8760
latitude : 105
longitude : 237

Değişkenler:
u10
v10
d2m
t2m
sp
tp
number
valid_time
latitude
longitude
expver

'temperature' isminde bir değişken bulunamadı.


In [3]:
import boto3
import io
import xarray as xr
from tempfile import NamedTemporaryFile

# S3 client oluşturma
s3 = boto3.client('s3')

# Bucket ve dosya anahtarını (key) belirtin
bucket_name = "airlinessss"
file_key = "2017_merged.nc"

# S3'ten dosyayı çekme
obj = s3.get_object(Bucket=bucket_name, Key=file_key)
data = obj['Body'].read()

# Geçici dosya oluşturup, veriyi oraya yazma
with NamedTemporaryFile(suffix=".nc") as tmp:
    tmp.write(data)
    tmp.flush()
    # NetCDF4 motorunu kullanarak dosyayı açma
    ds = xr.open_dataset(tmp.name, engine="netcdf4")

    # Dataset'in genel yapısını ve değişkenlerini görüntüleme
    print("Dataset:")
    print(ds)

    # Örneğin t2m (2m temperature) değişkeni için
    if 't2m' in ds:
        # t2m DataArray'ini pandas Series'e dönüştürmek için to_series() metodu kullanılabilir.
        # Bu, varsayılan olarak DataArray'in tüm değerlerini 1 boyutlu bir Series'e çevirir.
        t2m_series = ds['t2m'].to_series()
        t2m_stats = t2m_series.describe()
        print("2m Temperature istatistikleri:")
        print(t2m_stats)
    else:
        print("'t2m' değişkeni veri setinde bulunamadı.")

    # Aynı şekilde u10 için:
    if 'u10' in ds:
        u10_series = ds['u10'].to_series()
        u10_stats = u10_series.describe()
        print("10m u-component of wind istatistikleri:")
        print(u10_stats)
    else:
        print("'u10' değişkeni veri setinde bulunamadı.")

    # Zaman boyutu üzerinden ortalama değerleri hesaplamak
    if 'valid_time' in ds.dims:
        mean_values = ds.mean(dim='valid_time')
        print("Zaman boyutuna göre ortalama değerler:")
        print(mean_values)
    else:
        print("Zaman boyutu 'valid_time' bulunamadı.")

    ds.close()


Dataset:
<xarray.Dataset> Size: 5GB
Dimensions:     (valid_time: 8760, latitude: 105, longitude: 237)
Coordinates:
    number      int64 8B ...
  * valid_time  (valid_time) datetime64[ns] 70kB 2017-01-01 ... 2017-12-31T23...
  * latitude    (latitude) float64 840B 50.0 49.75 49.5 ... 24.5 24.25 24.0
  * longitude   (longitude) float64 2kB -125.0 -124.8 -124.5 ... -66.25 -66.0
    expver      (valid_time) <U4 140kB ...
Data variables:
    u10         (valid_time, latitude, longitude) float32 872MB ...
    v10         (valid_time, latitude, longitude) float32 872MB ...
    d2m         (valid_time, latitude, longitude) float32 872MB ...
    t2m         (valid_time, latitude, longitude) float32 872MB ...
    sp          (valid_time, latitude, longitude) float32 872MB ...
    tp          (valid_time, latitude, longitude) float32 872MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:         

In [4]:
# valid_time değişkeninin değerlerini yazdırma
print("valid_time içeriği:")
print(ds['valid_time'].values)
print(ds['valid_time'])


valid_time içeriği:
['2017-01-01T00:00:00.000000000' '2017-01-01T01:00:00.000000000'
 '2017-01-01T02:00:00.000000000' ... '2017-12-31T21:00:00.000000000'
 '2017-12-31T22:00:00.000000000' '2017-12-31T23:00:00.000000000']
<xarray.DataArray 'valid_time' (valid_time: 8760)> Size: 70kB
array(['2017-01-01T00:00:00.000000000', '2017-01-01T01:00:00.000000000',
       '2017-01-01T02:00:00.000000000', ..., '2017-12-31T21:00:00.000000000',
       '2017-12-31T22:00:00.000000000', '2017-12-31T23:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
    number      int64 8B ...
  * valid_time  (valid_time) datetime64[ns] 70kB 2017-01-01 ... 2017-12-31T23...
    expver      (valid_time) <U4 140kB ...
Attributes:
    long_name:      time
    standard_name:  time
